Used for comparison against SPY ETF.

In [29]:
# Dependencies
%pip install mplfinance
import requests
import json
from pymongo import MongoClient
import mplfinance as mpf

In [30]:
# Import the API key securel
# Sorry, I initially meant vantage..
from kirsten_api_keys import alpha_advantage_api

In [31]:
# Define the market index symbol (S&P 500 ETF - SPY)
symbol = "SPY"

In [32]:
# Construct the API request URL to fetch daily adjusted closing prices
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SPY&outputsize=full&apikey=alpha_advantage_api'

In [33]:
# Send a GET request to the Alpha Vantage API to fetch market data
r = requests.get(url)

In [34]:
# Parse the JSON response from the API
data = r.json()

In [39]:
# Rename it to market_index_spy_data
market_index_spy_data = data  # Assign API response to new variable

# Verify the rename
print(type(market_index_spy_data))  # Should be dict or list

<class 'dict'>


In [ ]:
# Ensure MongoDB is connected
mongo = MongoClient("mongodb://localhost:27017/")

# Select the database
db = mongo["project3_group4"]

# Select the collection
market_index_collection = db["market_index_spy_data"]

# Extract the time series data from the API response
if "Time Series (Daily)" in market_index_spy_data:
    time_series_data = market_index_spy_data["Time Series (Daily)"]

    # Transform it into a list of individual daily records
    formatted_data = []
    for date, values in time_series_data.items():
        formatted_data.append({
            "date": date,  # Store the date as a string
            "open": float(values["1. open"]),
            "high": float(values["2. high"]),
            "low": float(values["3. low"]),
            "close": float(values["4. close"]),
            "volume": int(values["5. volume"]),
            "symbol": "SPY" 
        })

    # Insert the formatted data into MongoDB
    market_index_collection.insert_many(formatted_data)

    print("Data inserted successfully into market_index_spy_data collection")
else:
    print("Error: Unexpected API response")

Data inserted successfully into market_index_spy_data collection!


In [41]:
# List all collections in the database
print(db.list_collection_names())  # Should include "market_index_spy_data"

['stock_data', 'market_index_spy_data']


In [42]:
# Print a few documents from the collection
sample_data = list(market_index_collection.find().limit(3))
print(sample_data)

[{'_id': ObjectId('679c31222eb19b317a5c1a51'), 'date': '2025-01-30', 'open': 603.96, 'high': 606.6, 'low': 600.715, 'close': 605.04, 'volume': 36575953, 'symbol': 'SPY'}, {'_id': ObjectId('679c31222eb19b317a5c1a52'), 'date': '2025-01-29', 'open': 603.72, 'high': 604.13, 'low': 599.22, 'close': 601.81, 'volume': 37177429, 'symbol': 'SPY'}, {'_id': ObjectId('679c31222eb19b317a5c1a53'), 'date': '2025-01-28', 'open': 600.62, 'high': 605.37, 'low': 597.25, 'close': 604.52, 'volume': 44433322, 'symbol': 'SPY'}]
